In [0]:
from pyspark.sql import functions as F

# 1) Ler Bronze
bronze = spark.table("workspace.pessoas.bronze_pessoas")

# 2) Renomear colunas
rename_map = {
    "Codigo": "codigo",
    "PrimeiroNome": "primeiro_nome",
    "UltimoNome": "ultimo_nome",
    "email": "email",
    "Genero": "genero",
    "Avatar": "avatar",
    "Pais": "pais",
    "Nascimento": "nascimento"
}

df = bronze
for old, new in rename_map.items():
    if old in df.columns:
        df = df.withColumnRenamed(old, new)

# 3) Data dd/MM/yyyy -> date
df = df.withColumn("nascimento", F.to_date(F.col("nascimento"), "dd/MM/yyyy"))

# 4) Trim em strings
types = dict(df.dtypes)
df = df.select([F.trim(F.col(c)).alias(c) if types.get(c)=="string" else F.col(c) for c in df.columns])

# 5) Tabela Silver (Delta gerenciada)
spark.sql("DROP TABLE IF EXISTS workspace.pessoas.silver_pessoas")
df.write.mode("overwrite").format("delta").saveAsTable("workspace.pessoas.silver_pessoas")

# 6) Conferir
spark.sql("SELECT * FROM workspace.pessoas.silver_pessoas LIMIT 10").show()
